## Initial setup

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *

In [3]:
bs = 256

In [4]:
data_path = Config.data_path()
lang = 'nl'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

## Download wikipedia data

In [5]:
from nlputils import split_wiki,get_wiki

In [6]:
get_wiki(path, lang)

/storage/nlwiki/nlwiki already exists; not downloading


In [7]:
path.ls()

[PosixPath('/storage/nlwiki/wikiextractor'),
 PosixPath('/storage/nlwiki/nlwiki-latest-pages-articles.xml'),
 PosixPath('/storage/nlwiki/nlwiki-latest-pages-articles-small.xml.bz2'),
 PosixPath('/storage/nlwiki/.ipynb_checkpoints'),
 PosixPath('/storage/nlwiki/nlwiki'),
 PosixPath('/storage/nlwiki/nlwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/storage/nlwiki/nlwiki-latest-pages-articles-small.xml'),
 PosixPath('/storage/nlwiki/docs'),
 PosixPath('/storage/nlwiki/log')]

### Split in separate files

In [8]:
dest = split_wiki(path, lang)

/storage/nlwiki/docs already exists; not splitting


In [9]:
dest.ls()[:5]

[PosixPath('/storage/nlwiki/docs/Theophanu.txt'),
 PosixPath('/storage/nlwiki/docs/Witte dovenetel.txt'),
 PosixPath('/storage/nlwiki/docs/Minskoffensief.txt'),
 PosixPath('/storage/nlwiki/docs/Cochabamba (stad).txt'),
 PosixPath('/storage/nlwiki/docs/Joel Osteen.txt')]

## Create databunch for language model

In [10]:
data = (TextList.from_folder(dest)
       .split_by_rand_pct(0.1, seed=42)
       .label_for_lm()
       .databunch(bs=bs, num_workers=1))
data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(60000, 183926)

In [11]:
# data.train_ds[:5]

## Train language model

In [10]:
data = load_data(dest, f'{lang}_databunch', bs=bs, num_workers=1)

In [11]:
# data.train_ds[:1]

In [12]:
# learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()
learn = language_model_learner(data, AWD_LSTM, drop_mult=1.0, pretrained=False)

In [13]:
learn.unfreeze()

In [14]:
# learn.lr_find()
# learn.recorder.plot()

In [15]:
lr = 1e-2

In [16]:
# learn.fit_one_cycle(1, lr, moms=(0.8, 0.7))

In [17]:
learn.fit_one_cycle(10, lr, moms=(0.8, 0.7))
# learn.fit_one_cycle(4, lr, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.524150,3.377209,0.373016,2:02:17
1,3.460805,3.315189,0.375193,2:02:05
2,3.445260,3.289851,0.378650,2:02:03
3,3.404821,3.240744,0.382945,2:01:59
4,3.363085,3.197753,0.387393,2:02:03
5,3.308877,3.138483,0.393465,2:02:04
6,3.250464,3.077751,0.399487,2:02:05
7,3.184076,3.016916,0.406360,2:02:02
8,3.162227,2.972598,0.411225,2:02:06
9,3.130161,2.958975,0.412856,2:02:06


In [18]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
# learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [19]:
TEXT = "In zijn laatste roman"
N_WORDS = 40
N_SENTENCES = 2

print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In zijn laatste roman 
 
  In de Nederlandse vertaling van " De Zeven Koningen " ( Nederlands : " Het huis van de koning " ) is een Nederlandse roman uit 1977 van de Nederlandse
In zijn laatste roman ( De ban van de ring ) 
 
  In de ban is een roman van de Nederlandse schrijver Maarten Thijssen . Het verhaal werd in 1956 door Gerard Reve verfilmd en is


In [20]:
learn.predict?

In [33]:
TEXT = "Jan Wolkers was afkomstig uit een gereformeerd milieu waar hij in zijn tienerjaren afstand van heeft genomen. De ouders van Wolkers (Jan Hendrik Wolkers sr. 1890-1976 en Jannetje van der Heijde 1899-1979) kwamen oorspronkelijk uit Amsterdam en hadden in totaal elf kinderen, van wie Jan het derde was. Vader Wolkers bezat voor de Tweede Wereldoorlog een slecht lopende kruidenierswinkel in Oegstgeest."
N_WORDS = 500
N_SENTENCES = 1

print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.6) for _ in range(N_SENTENCES)))

Jan Wolkers was afkomstig uit een gereformeerd milieu waar hij in zijn tienerjaren afstand van heeft genomen. De ouders van Wolkers (Jan Hendrik Wolkers sr. 1890-1976 en Jannetje van der Heijde 1899-1979) kwamen oorspronkelijk uit Amsterdam en hadden in totaal elf kinderen, van wie Jan het derde was. Vader Wolkers bezat voor de Tweede Wereldoorlog een slecht lopende kruidenierswinkel in Oegstgeest. In zijn jeugd was hij al op jonge leeftijd betrokken bij de Nederlandse Hervormde Kerk . Na zijn eindexamen ging hij naar de Katholieke Universiteit Delft , waar hij in 1954 de " Theologische School " studeerde . In 1954 werd hij predikant te Gouda . 
 
  In 1964 werd hij predikant in Leiden , waar hij van 1966 tot 1978 werkzaam was . In 1981 werd hij predikant in Vlaardingen , waar hij onder meer als predikant werkzaam was . 
 
  Na zijn afstuderen was Hij redacteur van de VARA - gids " Het Parool " , waarin hij tot 1988 redacteur was . Vanaf 1975 was hij redacteur van het tijdschrift " De 